In [1]:
# the below code is from DAE course notebook, so it needs to be altered to avoid plagiarism

import numpy as np  # import auxiliary library, typical idiom
import pandas as pd  # import the Pandas library, typical idiom

# next command ensures that plots appear inside the notebook
# %matplotlib inline
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns  # also improves the look of plots
# sns.set()
# plt.rcParams['figure.figsize'] = 10, 5  # default hor./vert. size of plots, in inches
# plt.rcParams['lines.markeredgewidth'] = 1  # to fix issue with seaborn box plots; needed after import seaborn

# # reveal a hint only while holding the mouse down
# from IPython.display import HTML
# HTML("<style>.h,.c{display:none}.t{color:#296eaa}.t:active+.h{display:block;}</style>")

In [2]:
from datetime import datetime
from numba import jit

In [3]:
# the below code is from DAE course notebook, so it needs to be altered to avoid plagiarism

from sklearn.linear_model import LinearRegression  # for linear regression
from sklearn.cluster import KMeans  # for clustering
from sklearn.tree import DecisionTreeClassifier  # for decision tree mining
from sklearn.metrics import mean_absolute_error, confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
#from treeviz import tree_print  # to print decision tree

In [4]:
file_export = 'export2012.csv'

In [5]:
data = pd.read_csv(file_export)

In [6]:
data.head(10)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112
5,173688,A_ACCEPTED,2011/10/01 11:42:43.308,2011/10/01 11:42:43.308,20000,2011/10/01 00:38:44.546,10862
6,173688,O_SELECTED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862
7,173688,A_FINALIZED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862
8,173688,O_CREATED,2011/10/01 11:45:11.197,2011/10/01 11:45:11.197,20000,2011/10/01 00:38:44.546,10862
9,173688,O_SENT,2011/10/01 11:45:11.380,2011/10/01 11:45:11.380,20000,2011/10/01 00:38:44.546,10862


In [7]:
# df = data.to_csv('export2012.csv') # allows to export the dataframe as a csv file

In [8]:
data.describe()

,case,AMOUNT_REQ,org:resource
count,190827.000000,190827.000000,190827.000000
mean,193851.453673,15438.929088,8271.081299
std,11688.517781,12315.133412,4700.713383
min,173688.000000,0.000000,112.000000
25%,183292.000000,6000.000000,10138.000000
50%,194242.000000,10500.000000,10913.000000
75%,203850.000000,20000.000000,11121.000000
max,214376.000000,99999.000000,11339.000000


In [9]:
data_sorted = data.sort_values(by=['case', 'event','completeTime'])

In [10]:
data_sorted.head(10)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource
5,173688,A_ACCEPTED,2011/10/01 11:42:43.308,2011/10/01 11:42:43.308,20000,2011/10/01 00:38:44.546,10862
20,173688,A_ACTIVATED,2011/10/13 10:37:29.226,2011/10/13 10:37:29.226,20000,2011/10/01 00:38:44.546,10629
18,173688,A_APPROVED,2011/10/13 10:37:29.226,2011/10/13 10:37:29.226,20000,2011/10/01 00:38:44.546,10629
7,173688,A_FINALIZED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112
17,173688,A_REGISTERED,2011/10/13 10:37:29.226,2011/10/13 10:37:29.226,20000,2011/10/01 00:38:44.546,10629
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112
19,173688,O_ACCEPTED,2011/10/13 10:37:29.226,2011/10/13 10:37:29.226,20000,2011/10/01 00:38:44.546,10629


In [11]:
df = data.copy()

# convert to time format the string type time. This will allow to calculate the time duration of the task.
df["startTime"] = pd.to_datetime(df["startTime"])
df["completeTime"] = pd.to_datetime(df["completeTime"])
df["duration"] = df["completeTime"] - df["startTime"]
df

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,duration
0,173688,A_SUBMITTED,2011-10-01 00:38:44.546,2011-10-01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,0 days 00:00:00
1,173688,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880,2011-10-01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,0 days 00:00:00
2,173688,A_PREACCEPTED,2011-10-01 00:39:37.906,2011-10-01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,0 days 00:00:00
3,173688,A_PREACCEPTED,2011-10-01 00:39:37.906,2011-10-01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,0 days 00:00:00
4,173688,W_Completeren aanvraag,2011-10-01 11:36:46.437,2011-10-01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,0 days 00:08:27.480000
...,...,...,...,...,...,...,...,...
190822,214376,A_SUBMITTED,2012-02-29 23:51:16.799,2012-02-29 23:51:16.799,15000,2012/02/29 23:51:16.799,112,0 days 00:00:00
190823,214376,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423,2012-02-29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,0 days 00:00:00
190824,214376,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423,2012-02-29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,0 days 00:00:00
190825,214376,W_Afhandelen leads,2012-03-01 09:26:46.736,2012-03-01 09:27:41.325,15000,2012/02/29 23:51:16.799,11169,0 days 00:00:54.589000


In [12]:
#datetime.strptime(data["completeTime"], '%Y-%m-%d')
#print(datetime)

In [13]:
data['startTime'] == data["completeTime"]

0          True
1          True
2          True
3          True
4         False
          ...  
190822     True
190823     True
190824     True
190825    False
190826     True
Length: 190827, dtype: bool

The above shows that the startTime and completeTime are not always the same.

In [14]:
data_sorted2 = data.sort_values(by=['AMOUNT_REQ'])

In [15]:
data_sorted2

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource
91296,193378,W_Afhandelen leads,2011/12/14 09:19:24.585,2011/12/14 09:28:18.936,0,2011/12/13 21:29:24.105,11189
91300,193378,A_CANCELLED,2011/12/14 09:43:25.393,2011/12/14 09:43:25.393,0,2011/12/13 21:29:24.105,10779
91299,193378,W_Afhandelen leads,2011/12/14 09:43:25.385,2011/12/14 09:43:25.385,0,2011/12/13 21:29:24.105,11189
91298,193378,W_Afhandelen leads,2011/12/14 09:32:45.306,2011/12/14 09:35:13.883,0,2011/12/13 21:29:24.105,11189
91293,193378,A_SUBMITTED,2011/12/13 21:29:24.106,2011/12/13 21:29:24.106,0,2011/12/13 21:29:24.105,112
...,...,...,...,...,...,...,...
179483,211643,A_SUBMITTED,2012/02/22 08:07:14.767,2012/02/22 08:07:14.767,99999,2012/02/22 08:07:14.767,112
49046,183576,A_SUBMITTED,2011/11/09 19:47:51.883,2011/11/09 19:47:51.883,99999,2011/11/09 19:47:51.882,112
49048,183576,A_DECLINED,2011/11/09 19:47:54.236,2011/11/09 19:47:54.236,99999,2011/11/09 19:47:51.882,112
179486,211643,A_PREACCEPTED,2012/02/22 08:07:58.846,2012/02/22 08:07:58.846,99999,2012/02/22 08:07:14.767,112


In [16]:
# data["next_task"] = calculator_nb(data)

data_iterrated = data.copy()
# for i in data_iterrated.iterrows():
#     if (i[0]+1 >= len(data)):
#         break
#     data["next_task"] = data.at[i[0]+1, 'case'] if (data.at[i[0] + 1, "case"] == i[1]["case"]) else 'null'

@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        # print(idx)
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]
        # res[idx] = event[idx + 1]  if (case[idx + 1] != case[idx])
        # else:
        #     res[idx] = 0

        idx+=1
    return res


# print(data_iterrated['case'].array)
data_iterrated["next_task"] = calculator_nb(data_iterrated['case'].values, data_iterrated["event"].values)

C:\Users\chbak\AppData\Local\Temp/ipykernel_1704/687891360.py:9: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_1704\687891360.py", line 11:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\chbak\AppData\Local\Temp/ipykernel_1704/687891360.py:9: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_1704\687891360.py", line 11:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_nb" was compiled in object mode without forceobj=

In [17]:
data_iterrated.head(40)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,A_PARTLYSUBMITTED
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,W_Completeren aanvraag
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,A_ACCEPTED
5,173688,A_ACCEPTED,2011/10/01 11:42:43.308,2011/10/01 11:42:43.308,20000,2011/10/01 00:38:44.546,10862,O_SELECTED
6,173688,O_SELECTED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862,A_FINALIZED
7,173688,A_FINALIZED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862,O_CREATED
8,173688,O_CREATED,2011/10/01 11:45:11.197,2011/10/01 11:45:11.197,20000,2011/10/01 00:38:44.546,10862,O_SENT
9,173688,O_SENT,2011/10/01 11:45:11.380,2011/10/01 11:45:11.380,20000,2011/10/01 00:38:44.546,10862,O_SENT


In [18]:
data_iterrated.head()

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,A_PARTLYSUBMITTED
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,W_Completeren aanvraag
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,A_ACCEPTED


In [19]:
list_of_events = data_iterrated["event"].unique()
event_to_num = {}
i=0
for event in list_of_events:
    event_to_num[str(event)] = i
    i += 1

In [20]:
event_to_num['None'] = 23

In [21]:
event_to_num

{'A_SUBMITTED': 0,
 'A_PARTLYSUBMITTED': 1,
 'A_PREACCEPTED': 2,
 'W_Completeren aanvraag': 3,
 'A_ACCEPTED': 4,
 'O_SELECTED': 5,
 'A_FINALIZED': 6,
 'O_CREATED': 7,
 'O_SENT': 8,
 'W_Nabellen offertes': 9,
 'O_SENT_BACK': 10,
 'W_Valideren aanvraag': 11,
 'A_REGISTERED': 12,
 'A_APPROVED': 13,
 'O_ACCEPTED': 14,
 'A_ACTIVATED': 15,
 'O_CANCELLED': 16,
 'A_DECLINED': 17,
 'A_CANCELLED': 18,
 'W_Afhandelen leads': 19,
 'O_DECLINED': 20,
 'W_Nabellen incomplete dossiers': 21,
 'W_Beoordelen fraude': 22,
 'None': 23}

In [22]:
data_iterrated_two = data_iterrated["event"].copy()
data_iterrated_two

0                    A_SUBMITTED
1              A_PARTLYSUBMITTED
2                  A_PREACCEPTED
3                  A_PREACCEPTED
4         W_Completeren aanvraag
                   ...          
190822               A_SUBMITTED
190823         A_PARTLYSUBMITTED
190824         A_PARTLYSUBMITTED
190825        W_Afhandelen leads
190826                A_DECLINED
Name: event, Length: 190827, dtype: object

In [23]:
for i in range(len(data_iterrated_two)):
    data_iterrated_two[i] = event_to_num[data_iterrated_two[i]]

In [24]:
event_to_num

{'A_SUBMITTED': 0,
 'A_PARTLYSUBMITTED': 1,
 'A_PREACCEPTED': 2,
 'W_Completeren aanvraag': 3,
 'A_ACCEPTED': 4,
 'O_SELECTED': 5,
 'A_FINALIZED': 6,
 'O_CREATED': 7,
 'O_SENT': 8,
 'W_Nabellen offertes': 9,
 'O_SENT_BACK': 10,
 'W_Valideren aanvraag': 11,
 'A_REGISTERED': 12,
 'A_APPROVED': 13,
 'O_ACCEPTED': 14,
 'A_ACTIVATED': 15,
 'O_CANCELLED': 16,
 'A_DECLINED': 17,
 'A_CANCELLED': 18,
 'W_Afhandelen leads': 19,
 'O_DECLINED': 20,
 'W_Nabellen incomplete dossiers': 21,
 'W_Beoordelen fraude': 22,
 'None': 23}

In [25]:
next_task = data_iterrated["next_task"].copy()
for i in range(len(next_task)):
    #print(next_task[i])
    try:
        next_task[i] = event_to_num[next_task[i]]
    except:
        next_task[i] = 23

In [26]:
#data_iterrated_two["next_task"] = next_task
data_to_lr = data_iterrated.copy()
data_to_lr["event"] = data_iterrated_two
data_to_lr["next_task"] = next_task
data_to_lr

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task
0,173688,0,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,1
1,173688,1,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,2
2,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,2
3,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,3
4,173688,3,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,4
...,...,...,...,...,...,...,...,...
190822,214376,0,2012/02/29 23:51:16.799,2012/02/29 23:51:16.799,15000,2012/02/29 23:51:16.799,112,1
190823,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,1
190824,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,19
190825,214376,19,2012/03/01 09:26:46.736,2012/03/01 09:27:41.325,15000,2012/02/29 23:51:16.799,11169,17


In [27]:
x = data_to_lr[["event"]]
y = data_to_lr["next_task"]

In [28]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
naive_predictor = LinearRegression()
naive_predictor.fit(X_train, y_train)
naive_predictor.score(X_test, y_test)
prediction = naive_predictor.predict(data_to_lr[["event"]]).round()

In [29]:
result_data = data_to_lr.copy()
result_data["prediction"] = prediction
result_data["prediction"] = result_data["prediction"].astype(int)

In [30]:
result_data

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,prediction
0,173688,0,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,1,3
1,173688,1,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,2,4
2,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,2,5
3,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,3,5
4,173688,3,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,4,6
...,...,...,...,...,...,...,...,...,...
190822,214376,0,2012/02/29 23:51:16.799,2012/02/29 23:51:16.799,15000,2012/02/29 23:51:16.799,112,1,3
190823,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,1,4
190824,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,19,4
190825,214376,19,2012/03/01 09:26:46.736,2012/03/01 09:27:41.325,15000,2012/02/29 23:51:16.799,11169,17,18


In [31]:
num_to_event = {v: k for k, v in event_to_num.items()}
num_to_event

{0: 'A_SUBMITTED',
 1: 'A_PARTLYSUBMITTED',
 2: 'A_PREACCEPTED',
 3: 'W_Completeren aanvraag',
 4: 'A_ACCEPTED',
 5: 'O_SELECTED',
 6: 'A_FINALIZED',
 7: 'O_CREATED',
 8: 'O_SENT',
 9: 'W_Nabellen offertes',
 10: 'O_SENT_BACK',
 11: 'W_Valideren aanvraag',
 12: 'A_REGISTERED',
 13: 'A_APPROVED',
 14: 'O_ACCEPTED',
 15: 'A_ACTIVATED',
 16: 'O_CANCELLED',
 17: 'A_DECLINED',
 18: 'A_CANCELLED',
 19: 'W_Afhandelen leads',
 20: 'O_DECLINED',
 21: 'W_Nabellen incomplete dossiers',
 22: 'W_Beoordelen fraude',
 23: 'None'}

In [32]:
predicted_event_str = result_data["prediction"].copy()
for i in range(len(predicted_event_str)):
    predicted_event_str[i] = num_to_event[predicted_event_str[i]]

In [33]:
predicted_event_str

0         W_Completeren aanvraag
1                     A_ACCEPTED
2                     O_SELECTED
3                     O_SELECTED
4                    A_FINALIZED
                   ...          
190822    W_Completeren aanvraag
190823                A_ACCEPTED
190824                A_ACCEPTED
190825               A_CANCELLED
190826                A_DECLINED
Name: prediction, Length: 190827, dtype: object

In [34]:
result_data_str = data_iterrated.copy()
result_data_str["prediction"] = predicted_event_str

In [35]:
result_data_str["duration"] = pd.to_datetime(result_data_str["completeTime"]) - pd.to_datetime(result_data_str["startTime"])
result_data_str.head(40)



,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,prediction,duration
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,A_PARTLYSUBMITTED,W_Completeren aanvraag,0 days 00:00:00
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,A_ACCEPTED,0 days 00:00:00
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,O_SELECTED,0 days 00:00:00
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,W_Completeren aanvraag,O_SELECTED,0 days 00:00:00
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,A_ACCEPTED,A_FINALIZED,0 days 00:08:27.480000
5,173688,A_ACCEPTED,2011/10/01 11:42:43.308,2011/10/01 11:42:43.308,20000,2011/10/01 00:38:44.546,10862,O_SELECTED,A_FINALIZED,0 days 00:00:00
6,173688,O_SELECTED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862,A_FINALIZED,O_CREATED,0 days 00:00:00
7,173688,A_FINALIZED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862,O_CREATED,O_SENT,0 days 00:00:00
8,173688,O_CREATED,2011/10/01 11:45:11.197,2011/10/01 11:45:11.197,20000,2011/10/01 00:38:44.546,10862,O_SENT,W_Nabellen offertes,0 days 00:00:00
9,173688,O_SENT,2011/10/01 11:45:11.380,2011/10/01 11:45:11.380,20000,2011/10/01 00:38:44.546,10862,O_SENT,O_SENT_BACK,0 days 00:00:00


In [36]:

# train = result_data_str[result_data_str.index <= len(result_data_str) / 1.2]
# y = train['duration']
# x = train[]
# ARMAmodel = SARIMAX(y, order = (1, 0, 1))


# test = result_data_str[result_data_str.index > len(result_data_str) / 1.2]

# ARMAmodel = ARMAmodel.fit()

# y_pred = ARMAmodel.get_forecast(len(test.index))
# y_pred_df = y_pred.conf_int()#alpha = 0.05) 
# y_pred_df["Predictions"] = ARMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])
# y_pred_df.index = test.index
# y_pred_out = y_pred_df["Predictions"] 

events_count = result_data_str.groupby("event")['duration'].agg('count')
event_duration_sum = result_data_str.groupby("event")['duration'].agg('sum')

duration_per_event = event_duration_sum / events_count

# for i in range(len(result_data_str)):
#     print(result_data_str.at[i])# = duration_per_event[result_data_str[i]["event"]]

@jit(parallel = True)
def make_prediction(event, dict):
    res = np.empty(len(event), dtype=object)
    # idx = 0
    for idx,_ in enumerate(event):
        # print(idx)       
        res[idx] = dict[event[idx]]
        # res[idx] = event[idx + 1]  if (case[idx + 1] != case[idx])
        # else:
        #     res[idx] = 0

        # idx+=1
    return res

result_data_str["duration_prediction"] = make_prediction(result_data_str['event'].values, duration_per_event)




C:\Users\chbak\AppData\Local\Temp/ipykernel_1704/4121750575.py:25: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "make_prediction" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_1704\4121750575.py", line 27:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\chbak\AppData\Local\Temp/ipykernel_1704/4121750575.py:25: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "make_prediction" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_1704\4121750575.py", line 27:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "make_prediction" was compiled in object mode witho

In [37]:
result_data_str

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,prediction,duration,duration_prediction
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,A_PARTLYSUBMITTED,W_Completeren aanvraag,0 days 00:00:00,0 days 00:00:00
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,A_ACCEPTED,0 days 00:00:00,0 days 00:00:00
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,O_SELECTED,0 days 00:00:00,0 days 00:00:00
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,W_Completeren aanvraag,O_SELECTED,0 days 00:00:00,0 days 00:00:00
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,A_ACCEPTED,A_FINALIZED,0 days 00:08:27.480000,0 days 00:10:00.791783229
...,...,...,...,...,...,...,...,...,...,...,...
190822,214376,A_SUBMITTED,2012/02/29 23:51:16.799,2012/02/29 23:51:16.799,15000,2012/02/29 23:51:16.799,112,A_PARTLYSUBMITTED,W_Completeren aanvraag,0 days 00:00:00,0 days 00:00:00
190823,214376,A_PARTLYSUBMITTED,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,A_PARTLYSUBMITTED,A_ACCEPTED,0 days 00:00:00,0 days 00:00:00
190824,214376,A_PARTLYSUBMITTED,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,W_Afhandelen leads,A_ACCEPTED,0 days 00:00:00,0 days 00:00:00
190825,214376,W_Afhandelen leads,2012/03/01 09:26:46.736,2012/03/01 09:27:41.325,15000,2012/02/29 23:51:16.799,11169,A_DECLINED,A_CANCELLED,0 days 00:00:54.589000,0 days 00:16:52.801100320


In [38]:
# VARMAX example
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

from skforecast.ForecasterAutoreg import ForecasterAutoreg # pip install skforecast

In [69]:
df_filtered = result_data[result_data["case"] < len(result_data)]
df_prediction = result_data[result_data["case"] >= len(result_data)]


df_filtered=df_filtered.dropna()
df_prediction=df_prediction.dropna()
df_filtered["startTime"] = pd.to_datetime(df_filtered["startTime"])
df_prediction["startTime"] = pd.to_datetime(df_prediction["startTime"])


df_filtered_new = [v for _, v in df_filtered.groupby('event')]
dictionary_event_startTime = np.empty(len(df_filtered_new), dtype=object) 
for idx, val in enumerate(df_filtered_new):
    dictionary_event_startTime[idx] = val["startTime"]

# listVal = df_filtered[["event", "startTime"]]
# forecaster = ForecasterAutoreg(
#                 regressor = RandomForestRegressor(),
#                 lags = 6
#              )


# forecaster.fit(y=listVal["event"])

# steps = 10
# predictions = forecaster.predict(steps=steps)
# print(predictions)
# dictionary_events = df_filtered["startTime"]#, dtype=float)[0:18]# np.asarray(
# random_forest = RandomForestClassifier(n_estimators=50,
#                                        max_depth=10, random_state=1)
# random_forest.fit([dictionary_event_startTime[0]], [dictionary_events[0]])
# rf_prediction = random_forest.predict()
# rfe = RFE(estimator=RandomForestRegressor())
# fit = rfe.fit([dictionary_event_startTime[:5]], dictionary_events[:5])
# y_pred = fit.predict([df_prediction["startTime"][:5]])

# y_pred[0]

# dictionary_events
# for i in :

#     data_exog = [x + random() for x in range(100)]
# # fit model
# model = VARMAX(data, exog=data_exog, order=(1, 1))


In [40]:
# X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)

# X

In [41]:
#OLS regression
import statsmodels.api as sm

In [81]:
y = np.floor(pd.to_numeric(df_filtered["startTime"].values), dtype=float)
X = np.array(df_filtered["event"].values, dtype=float)
olsmod = sm.OLS(y, X)
olsres = olsmod.fit()
ypred = olsres.predict(X)
Xnew = np.array(df_prediction["event"].values, dtype=float)
ynewpred = olsres.predict(Xnew) 
df_prediction["time_OLS"] = pd.to_datetime(ynewpred)

# olsres = olsmod.fit()



,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,prediction,time_OLS
79248,190827,0,2011-12-02 20:04:55.523,2011/12/02 20:04:55.523,10000,2011/12/02 20:04:55.523,112,1,3,1970-01-01 00:00:00.000000000
79249,190827,1,2011-12-02 20:04:56.361,2011/12/02 20:04:56.361,10000,2011/12/02 20:04:55.523,112,2,4,1973-03-13 00:54:21.340923792
79250,190827,2,2011-12-02 20:05:41.776,2011/12/02 20:05:41.776,10000,2011/12/02 20:04:55.523,112,2,5,1976-05-23 01:48:42.681847584
79251,190827,2,2011-12-02 20:05:41.776,2011/12/02 20:05:41.776,10000,2011/12/02 20:04:55.523,112,3,5,1976-05-23 01:48:42.681847584
79252,190827,3,2011-12-02 20:05:46.431,2011/12/02 20:20:08.125,10000,2011/12/02 20:04:55.523,112,4,6,1979-08-03 02:43:04.022771392
...,...,...,...,...,...,...,...,...,...,...
190822,214376,0,2012-02-29 23:51:16.799,2012/02/29 23:51:16.799,15000,2012/02/29 23:51:16.799,112,1,3,1970-01-01 00:00:00.000000000
190823,214376,1,2012-02-29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,1,4,1973-03-13 00:54:21.340923792
190824,214376,1,2012-02-29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,19,4,1973-03-13 00:54:21.340923792
190825,214376,19,2012-03-01 09:26:46.736,2012/03/01 09:27:41.325,15000,2012/02/29 23:51:16.799,11169,17,18,2030-09-16 17:12:45.477552128
